# **지능형 IoT 응용** (2021년 2학기)
#### 한국기술교육대학교 컴퓨터공학부 스마트 IoT 트랙


---

# 06. TensorFlow Lite for Microcontrollers를 이용한 애플리케이션 구축 및 배포

---

### Acknowledgement


이 자료는 다음 서적의 내용을 바탕으로 작성되었음
- 초소형 머신러닝 TinyML, 피트 워든, 대니얼 시투나야케 지음, 맹윤호, 임지순 옮김, 한빛미디어
  - 5장, 6장

---

## TinyML 'Hello World' 애플리케이션 구축

- 이전 내용에서 sin 함수 모델을 생성하는 코드를 살펴보고 모델을 생성하는 것을 직접 해보았음
- 그러나 모델은 머신러닝 애플리케이션의 일부이며, 모델 자체는 스스로 아무것도 할 수 없는 정보 덩어리임
- 모델을 사용하기 위해서는 모델을 실행하는 데 필요한 환경을 설정하고 모델에 입력을 제공하고, 모델의 출력을 사용하여 어떤 동작을 수행하는 애플리케이션 코드가 필요함
- 여기서는 이전에 구축한 사인 모델을 사용하여 간단히 LED를 점멸하게 만드는 임베디드 애플리케이션을 구현하는 것을 살펴볼 것임
  - x 값을 모델에 공급
  - 모델 추론 실행
  - 모델의 출력 결과를 사용하여 LED 점멸
  - 위 과정을 연속적으로 수행하는 애플리케이션


#### 기본적인 TinyML 애플리케이션의 아키텍처

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb2xWLR%2Fbtq6Vrj1ehU%2FwvuIUEas0EGcJkeyMhET5k%2Fimg.png">
<center> (이미지 출처: https://toitoitoi79.tistory.com/99) </center>

- 코드는 TFLM github 저장소에서 확인할 수 있음
  - 복잡한 로직을 피하면서 TinyML 애플리케이션의 전체 구현을 최소화하도록 설계된 C++ 11 프로그램
  - 이 예제 코드를 이해하면 TFLM 프로그램의 일반적인 구조를 이해하고 자신의 프로젝트에 동일한 구조를 재사용할 수 있게 될 것임


---

### Notice
- __본 수업에서는 TinyML 번역서의 한글 소스코드 저장소의 예제 코드를 기반으로 함__
  - github 저장소 - https://github.com/yunho0130/tensorflow-lite
  - TinyML 'Hellow World' 예제 위치 - https://github.com/yunho0130/tensorflow-lite/tree/master/tensorflow/lite/micro/examples/hello_world

- __현재 TFLM 저장소의 코드는 계속 업데이트 되고 있기 때문에 책의 코드와 다른 부분도 있음을 참고__
  - TFLM github 저장소 - https://github.com/tensorflow/tflite-micro

- __아래의 실행 가능 코드는 Colab에서 실행__

---



#### TFLM 소스 코드 다운받기

- git clone 명령어를 이용하여 저장소 코드를 복제함

- /content 디렉토리는 Colab 실행시 홈 디렉토리
  - Colab 실행시 기본으로 생성되어 있는 sample_data 디렉토리와 같은 경로 상에 디렉토리가 생성됨

In [1]:
%cd /content
%rm -rf tensorflow-lite
! git clone https://github.com/yunho0130/tensorflow-lite


/content
Cloning into 'tensorflow-lite'...
remote: Enumerating objects: 847199, done.
remote: Counting objects: 100% (40826/40826), done.
remote: Compressing objects: 100% (831/831), done.
remote: Total 847199 (delta 40437), reused 39995 (delta 39995), pack-reused 806373
Receiving objects: 100% (847199/847199), 596.14 MiB | 23.12 MiB/s, done.
Resolving deltas: 100% (678602/678602), done.
Updating files: 100% (19934/19934), done.


## 6.1 테스트 작성

- 테스트 코드
  - 모델을 로드하고 추론을 실행해 원하는 예측이 이루어지는지 확인하는 테스트 코드를 먼저 확인할 것임
  - 테스트 코드를 실행해봄으로써 구현하려는 애플리케이션 코드가 예상대로 작동한다는 것을 먼저 검증해볼 수 있음
  - hello_world_test.cc 파일: https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/hello_world/hello_world_test.cc
  - 애플리케이션의 핵심적인 코드를 파악할 수 있음

### 6.1.1 종속성 불러오기

```cpp
#include "tensorflow/lite/micro/examples/hello_world/sine_model_data.h"
#include "tensorflow/lite/micro/kernels/all_ops_resolver.h"
#include "tensorflow/lite/micro/micro_error_reporter.h"
#include "tensorflow/lite/micro/micro_interpreter.h"
#include "tensorflow/lite/micro/testing/micro_test.h"
#include "tensorflow/lite/schema/schema_generated.h"
```

- tensorflow/lite/micro/examples/hello_world/sine_model_data.h
  - C++ 코드로 변환한 사인 함수 모델 데이터 배열을 정의하는 헤더 파일

- tensorflow/lite/micro/kernels/all_ops_resolver.h
  - TFLM 인터프리터(모델 실행하는 역할)가 모델에서 사용하는 TensorFlow operation(텐서플로우에서 계산을 수행하는 핵심 노드)을 로드할 수 있게 하는 클래스

- tensorflow/lite/micro/micro_error_reporter.h
  - 디버깅을 위해 오류와 출력을 로깅하는 클래스

- tensorflow/lite/micro/micro_interpreter.h
  - 모델을 실행하는 마이크로컨트롤러용 텐서플로우 라이트 인터프리터 클래스

- tensorflow/lite/micro/testing/micro_test.h
  - 테스트 작성을 위한 간단한 프레임워크. 테스트를 위한 여러 가지 매크로가 정의되어 있음
  - 매크로: 코드 덩어리에 이름을 붙여서 다른 곳에 포함하여 재사용 가능하도록 정의한 것

- tensorflow/lite/schema/schema_generated.h
  - TensorFlow Lite FlatBuffer 데이터 구조를 정의하는 스키마


### 6.1.2 테스트 설정

```cpp
TF_LITE_MICRO_TESTS_BEGIN

TF_LITE_MICRO_TEST(LoadModelAndPerformInference) {
```

- TF_LITE_MICRO_TESTS_BEGIN, TF_LITE_MICRO_TEST
  - micro_test.h 헤더 파일에 정의된 매크로 이름




### 6.1.3 데이터 기록 준비

```cpp
  // Set up logging
  tflite::MicroErrorReporter micro_error_reporter;
  tflite::ErrorReporter* error_reporter = &micro_error_reporter;
```

- MicroErrorReporter 인스턴스 정의
  - micro_error_reporter.h에 정의되어 있으며 실행 중 디버그 정보를 기록하는 기능 제공


### 6.1.4 모델 매핑

```cpp
  // Map the model into a usable data structure. This doesn't involve any
  // copying or parsing, it's a very lightweight operation.
  const tflite::Model* model = ::tflite::GetModel(g_sine_model_data);
  if (model->version() != TFLITE_SCHEMA_VERSION) {
    TF_LITE_REPORT_ERROR(error_reporter,
                         "Model provided is schema version %d not equal "
                         "to supported version %d.\n",
                         model->version(), TFLITE_SCHEMA_VERSION);
  }
```

- Model 구조체의 포인터 변수(model) 생성
  - Model 구조체, GetModel() 메소드는 schema_generated.h에 정의됨
  - GetModel() 메소드는 Model 타입의 포인터 반환
  - 모델 데이터를 담고 있음
  - g_sine_model_data: 모델 데이터 배열

- 모델 버전 번호 확인
  - 모델 버전 번호를 검색하는 메소드(model->version())을 호출하여 현재 사용 중인 텐서플로우 라이트 라이브러리의 버전을 담은 TFLITE_SCHEMA_VERSION과 모델의 버전 번호를 비교
  - 숫자가 일치하면 모델이 호환되는 버전의 텐서플로우 라이트 변환기로 변환됐다는 의미

- 버전 번호가 일치하지 않으면 매크로를 통해 에러를 기록함
  - TF_LITE_REPORT_ERROR는 miro_error_reporter.h에서 참조하는 tensorflow/lite/core/api/error_reporter.h에 정의된 매크로
  - error_reporter의 Report() 메소드를 호출하며 printf()와 유사하게 동작
  - 두개의 %d 형식지정자 자리에 model->version(), TFLITE_SCHEMA_VERSION 값 출력




### 6.1.5 AllOpsResolver 생성

```cpp
  // This pulls in all the operation implementations we need
  tflite::ops::micro::AllOpsResolver resolver;
```

- AllOpsResolver 클래스 인스턴스 생성
  - AllOpsResolver 클래스는 all_ops_resolver.h에 정의된 클래스이며 마이크로컨트롤러용 텐서플로우 라이트 인터프리터가 TFLM에서 사용 가능한 모든 연산(Op)에 접근할 수 있도록 함

### 6.1.6 텐서 아레나 정의

```cpp
  // Create an area of memory to use for input, output, and intermediate arrays.
  // Finding the minimum value for your model may require some trial and error.
  const int tensor_arena_size = 2 * 1024;
  uint8_t tensor_arena[tensor_arena_size];
```

- 모델이 실행되는 동안 필요한 작업 메모리 영역 할당 필요
- 텐서 아레나(tensor arena): 모델의 입력, 출력, 중간 텐서를 저장하는 데 사용되는 메모리 영역
- 2048 바이트 8비트 int 배열을 할당함

- 텐서 아레나의 적절한 크기?
  - 모델 아키텍처마다 크기, 입력, 출력, 중간 텐서가 다르므로 텐서 아레나의 적절한 크기는 알기 어려움
  - 마이크로컨트롤러는 RAM이 제한되어 있으므로 가능한 한 텐서 아레나를 작게 유지하여 나머지 프로그램을 위한 공간을 확보할 필요가 있음
  - 시행착오를 통해 최적의 크기를 찾아가는 방법이 있음 : 큰 크기부터 시작해서 모델이 실행되지 않을 때까지 크기를 줄여가는 방식

### 6.1.7 인터프리터 생성

```cpp
  // Build an interpreter to run the model with
  tflite::MicroInterpreter interpreter(model, resolver, tensor_arena,
                                       tensor_arena_size, error_reporter);
  // Allocate memory from the tensor_arena for the model's tensors
  TF_LITE_MICRO_EXPECT_EQ(interpreter.AllocateTensors(), kTfLiteOk);
```

- MicroInterpreter 클래스의 인스턴스 생성
  - TFLM에서 모델을 실행하는 기능 수행
  - Model, AllOpsResolver 객체, tensor arena 배열 등을 생성자로 전달

- AllocateTensors() 메소드 호출
  - 모델이 정의한 모든 텐서를 확인한 후 tensor_arena에서 각 텐서로 메모리를 할당함
  - 추론 실행 전 반드시 호출 필요

- TF_LITE_MICRO_EXPECT_EQ 매크로
  - micro_test.h에 정의된 매크로
  - 2개의 인수의 값이 같은지 확인
  - 같지 않을 경우 로그 메시지를 출력하고 테스트가 실패한 것으로 됨




### 6.1.8 입력 텐서 검사

```cpp
  // Obtain a pointer to the model's input tensor
  TfLiteTensor* input = interpreter.input(0);

  // Make sure the input has the properties we expect
  TF_LITE_MICRO_EXPECT_NE(nullptr, input);
  // The property "dims" tells us the tensor's shape. It has one element for
  // each dimension. Our input is a 2D tensor containing 1 element, so "dims"
  // should have size 2.
  TF_LITE_MICRO_EXPECT_EQ(2, input->dims->size);
  // The value of each element gives the length of the corresponding tensor.
  // We should expect two single element tensors (one is contained within the
  // other).
  TF_LITE_MICRO_EXPECT_EQ(1, input->dims->data[0]);
  TF_LITE_MICRO_EXPECT_EQ(1, input->dims->data[1]);
  // The input is a 32 bit floating point value
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteFloat32, input->type);
```

- 모델의 입력 텐서에 대한 포인터를 얻기 위해 인터프리터의 input() 메소드 호출
  - 메소드의 인자: 원하는 텐서를 지정하는 인덱스
  - 이 예제에서는 입력 텐서가 하나만 있으므로 인덱스는 0

- TfLiteTensor
  - TensorFlow Lite에서 사용되는 텐서 구조체
  - tensorflow/lite/c/common.h에 정의됨
  - 아래 테스트 코드에서 이를 이용하여 텐서의 shape과 dimension, type을 확인함

- TF_LITE_MICRO_EXPECT_NE
  - TF_LITE_MICRO_EXPECT_EQ 매크로와 마찬가지로 micro_test.h에 정의된 매크로
  - 두 개 인수가 같지 않음을 확인
  - 같을 경우 로그 메시지를 출력하고 테스트가 실패한 것으로 됨

- TF_LITE_MICRO_EXPECT_NE(nullptr, input);
  - 입력 텐서(input)이 null pointer가 아닌지 확인

- TF_LITE_MICRO_EXPECT_EQ(2, input->dims->size);
  - 입력 텐서(input)의 dimension(dims) 크기가 2와 같은지 확인
  - 우리가 사용하는 모델의 입력은 스칼라 값이지만 케라스 레이어가 입력을 받아들이는 방식에 맞추려면 입력값을 하나의 숫자를 포함하는 2D 텐서로 만들어서 제공해야 함
  - 입력이 0인 경우 [[0]]이 되어야 함
  - 따라서 입력 데이터의 차원은 2가 되어야 함

- TF_LITE_MICRO_EXPECT_EQ(1, input->dims->data[0]);  TF_LITE_MICRO_EXPECT_EQ(1, input->dims->data[1]);
  - 텐서 구조가 올바른지 확인
  - dims의 data 변수는 각 차원마다 해당 차원의 크기를 나타내는 정수를 원소로 갖는 배열
  - 여기서 우리가 사용하는 입력의 형태는 각 차원에 하나의 원소를 포함하는 2차원 텐서이므로 dims의 data 배열의 0, 1번 원소는 모두 1이 되어야 함

- TF_LITE_MICRO_EXPECT_EQ(kTfLiteFloat32, input->type);
  - 입력 텐서의 type이 올바른지 확인
  - 여기서는 32비트 float(실수)임을 확인함


### 6.1.9 입력 데이터 제공 및 추론 실행

```cpp
  // Provide an input value
  input->data.f[0] = 0.;

  // Run the model and check that it succeeds
  TfLiteStatus invoke_status = interpreter.Invoke();
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, invoke_status);
```

- 입력 텐서에 추론에 사용할 입력 값 0.0 할당
  - input->data.f[0] = 0.;

- 인터프리터로 모델 실행
  - TfLiteStatus invoke_status = interpreter.Invoke();
  - Invoke() 메소드를 호출하면 모델이 실행되며, 입력 데이터를 이용하여 모델의 결과를 생성함
  - 모델 실행 결과는 모델의 출력 텐서에 저장됨
  - Invoke() 메소드는 TfLiteStatus 객체를 반환하여 실행의 성공 여부를 알려줌
  - 반환값은 kTfLiteOk 또는 kTfLiteError 중 하나
  - TF_LITE_MICRO_EXPECT_EQ 매크로를 통해 반환값이 kTfLiteOk인지 확인




### 6.1.10 출력 확인

```cpp
  // Obtain a pointer to the output tensor and make sure it has the
  // properties we expect. It should be the same as the input tensor.
  TfLiteTensor* output = interpreter.output(0);
  TF_LITE_MICRO_EXPECT_EQ(2, output->dims->size);
  TF_LITE_MICRO_EXPECT_EQ(1, output->dims->data[0]);
  TF_LITE_MICRO_EXPECT_EQ(1, output->dims->data[1]);
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteFloat32, output->type);
```

- 모델의 출력 텐서에 대한 포인터를 얻기 위해 interpreter의 output() 메소드 호출
  - 출력은 입력과 마찬가지로 2D 텐서 안에 포함된 32비트 실수 스칼라 값

- 테스트 프레임워크의 매크로를 이용해 출력 텐서의 차원, 크기, 자료형이 올바른지 확인



```cpp
  // Obtain the output value from the tensor
  float value = output->data.f[0];
  // Check that the output value is within 0.05 of the expected value
  TF_LITE_MICRO_EXPECT_NEAR(0., value, 0.05);
```

- 모델의 출력(value)이 정답치(sin(0.0))와 비슷한지 확인
  - TF_LITE_MICRO_EXPECT_NEAR 매크로
  - 첫 번째 인수(sin(0.0))와 두 번째 인수(value)의 차이가 세 번째 인수의 값(0.05)보다 작은지 확인


```cpp
  // Run inference on several more values and confirm the expected outputs
  input->data.f[0] = 1.;
  invoke_status = interpreter.Invoke();
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, invoke_status);

  value = output->data.f[0];
  TF_LITE_MICRO_EXPECT_NEAR(0.841, value, 0.05);

  input->data.f[0] = 3.;
  invoke_status = interpreter.Invoke();
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, invoke_status);

  value = output->data.f[0];
  TF_LITE_MICRO_EXPECT_NEAR(0.141, value, 0.05);

  input->data.f[0] = 5.;
  invoke_status = interpreter.Invoke();
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, invoke_status);

  value = output->data.f[0];
  TF_LITE_MICRO_EXPECT_NEAR(-0.959, value, 0.05);
```

- 이전의 테스트와 다른 입력 값에 대해 모델이 잘 작동하는지 추가로 검증하기 위해 추가적인 테스트 수행


### 6.1.11 테스트 실행

- 원래 목적은 마이크로컨트롤러에서 프로그램을 실행하는 것이지만 개발 시스템에서 코드를 빌드하고 실행할 수도 있음. 이를 통해 코드 작성과 디버그가 훨씬 쉬워짐
- 개인용 컴퓨터는 마이크로컨트롤러와 비교하여 출력을 로깅하고 코드를 검토하는데 편리한 도구를 제공하므로 버그를 훨씬 쉽게 파악할 수 있음
- 마이크로컨트롤러 기기에 코드를 배포하고 실행하는 데에도 시간이 걸리므로 개발 시스템 환경에서 코드를 실행해 보는 것이 훨씬 빠름


---
#### Notice
- 프로그램 빌드를 위해 Make를 사용하는데 Windows 환경에서는 사용하기에 어려움이 있음
- 아래 코드는 Colab에서 실행됨
- 리눅스 혹은 맥 OS 기반의 컴퓨터에서 코드를 직접 다운 받아서 로컬에서 실행해볼 수 있을 것임



#### 테스트 프로그램 빌드

- Make
  - 소프트웨어 빌드 작업을 자동화하는 도구
  - 1976년부터 사용됨
  - 개발자는 Makefile이라는 빌드 파일을 작성하여 Make 에 코드를 빌드하고 실행하는 방법을 지시함
  - TFLM 용 Makefile은 micro/tools/make/Makefile에 정의되어 있음

- make 명령어
  - 사용한 Makefile 경로 지정
  - 빌드하려는 구성 요소인 target 지정: 빌드하고자 하는 대상은 hello_world_test임. 여기에 접두어 test_ 를 붙여줌

- 아래 명령어를 실행하면 필요한 라이브러리와 도구가 다운되고 테스트 파일과 필요한 라이브러리를 빌드하여 실행 파일을 생성
- 빌드가 완료되면 프로그램을 실행하여 결과가 출력됨
  - 아래와 같이 출력되면 테스트 프로그램이 실행되고 모든 테스트가 성공했음을 나타냄

```
Testing LoadModelAndPerformInference
1/1 tests passed
~~~ALL TESTS PASSED~~~
```

In [7]:
%cd /content/tensorflow-lite/

! make -f tensorflow/lite/micro/tools/make/Makefile test_hello_world_test

/content/tensorflow-lite
tensorflow/lite/micro/tools/make/download_and_extract.sh "https://github.com/google/gemmlowp/archive/719139ce755a0f31cbf1c37f7f98adcc7fc9f425.zip" "7e8191b24853d75de2af87622ad293ba" tensorflow/lite/micro/tools/make/downloads/gemmlowp  
downloading https://github.com/google/gemmlowp/archive/719139ce755a0f31cbf1c37f7f98adcc7fc9f425.zip
tensorflow/lite/micro/tools/make/download_and_extract.sh "https://storage.googleapis.com/mirror.tensorflow.org/github.com/google/flatbuffers/archive/v1.11.0.tar.gz" "02c64880acb89dbd57eebacfd67200d8" tensorflow/lite/micro/tools/make/downloads/flatbuffers  
downloading https://storage.googleapis.com/mirror.tensorflow.org/github.com/google/flatbuffers/archive/v1.11.0.tar.gz
tensorflow/lite/micro/tools/make/download_and_extract.sh "https://github.com/mborgerding/kissfft/archive/v130.zip" "438ba1fef5783cc5f5f201395cc477ca" tensorflow/lite/micro/tools/make/downloads/kissfft patch_kissfft 
downloading https://github.com/mborgerding/kissf

#### 테스트 프로그램 실행하기

- 아래 경로에서 빌드된 테스트 프로그램을 실행할 수 있음

In [8]:
%cd /content/tensorflow-lite/

! tensorflow/lite/micro/tools/make/gen/linux_x86_64/bin/hello_world_test

/content/tensorflow-lite
Testing LoadModelAndPerformInference
1/1 tests passed
~~~ALL TESTS PASSED~~~



## 6.2 프로젝트 파일 구조

- 애플리케이션 루트
  - tensorflow/lite/micro/examples/hello_world/
  - https://github.com/yunho0130/tensorflow-lite/tree/master/tensorflow/lite/micro/examples/hello_world

- 주요 파일
  - Makefile.inc: 테스트 코드인 hello_world_test와 기본 애플케이션 코드인 hello_world를 포함하여 애플리케이션 내의 빌드 대상에 대한 정보가 포함된 Makefile

  - constants.h, constants.cc: 프로그램에서 사용되는 주요 상수를 정의

  - hello_world_test.cc: 위에서 살펴본 테스트 프로그램

  - main.cc: 애플리케이션이 장치에 배포되면 가장 먼저 실행되는 프로그램 진입점

  - main_functions.h, main_functions.cc: 프로그램 실행에 필요한 초기화를 수행하는 setup() 함수와 프로그램의 핵심 로직을 반복하여 실행하는 loop() 함수를 정의하는 파일. main.cc에서 이 함수들을 호출함

  - output_handler.h, output_handler.cc: 모델에 의한 추론이 실행될 때마다 모델의 출력을 표시하는데 사용되는 함수를 정의하는 파일 쌍. output_handler.cc의 기본 구현은 결과를 화면에 출력함. 이 구현을 재정의하면 다른 장치에서 다르게 동작하도록 할 수 있음

  - output_handler_test.c: output_handler.h/cc 코드가 올바른지 확인하는 테스트 프로그램

- Arduino용 코드 저장소
  - arduino용 TFLM library 저장소가 별도로 생성됨
  - https://github.com/tensorflow/tflite-micro-arduino-examples
  - hellow_world 예제 경로: https://github.com/tensorflow/tflite-micro-arduino-examples/tree/main/examples/hello_world
  - arduino_main.cpp, arduino_constants.cpp, arduino_output_handler.cpp 파일이 아두이노에서 동작하도록 프로그램이 빌드되는 경우 사용되는 커스텀 버전 파일


## 6.3 애플리케이션 소스 코드



### 6.3.1 main.cc

- main() 함수
  - C++ 프로그램에서 프로그램의 시작이 되는 전역 함수
  - main.cc 파일에 정의되어 있음
  - 마이크로컨트롤러가 시작될 때마다 실행됨
  - main_functions.h에 정의된 setup(), loop() 함수를 이용

 ```cpp
 #include "tensorflow/lite/micro/examples/hello_world/main_functions.h"
// This is the default main used on systems that have the standard C entry
// point. Other devices (for example FreeRTOS or ESP32) that have different
// requirements for entry code (like an app_main function) should specialize
// this main.cc file in a target-specific subfolder.
int main(int argc, char* argv[]) {
    setup();
    while (true) {
      loop();
    }
}
```


### 6.3.2 main_functions.cc


#### 코드 주요 기능

- 전역 변수 초기화

- 모델 실행을 위한 준비: setup() 함수
  - Model 객체 생성
  - AllOpsResolver 객체 생성
  - Interpreter 객체 생성
  - 텐서 메모리 할당

- 모델 실행 및 결과 처리: loop() 함수
  - 모델 입력 준비
  - Interpreter 실행
  - 모델 출력 처리


#### main_functions.cc 코드 설명

- 필요한 헤더 파일 선언
  - 위에서 살펴본 테스트 프로그램에서 사용한 것과 유사

 ```cpp
#include "tensorflow/lite/micro/examples/hello_world/constants.h"
#include "tensorflow/lite/micro/examples/hello_world/output_handler.h"
#include "tensorflow/lite/micro/examples/hello_world/sine_model_data.h"
#include "tensorflow/lite/micro/kernels/all_ops_resolver.h"
#include "tensorflow/lite/micro/micro_error_reporter.h"
#include "tensorflow/lite/micro/micro_interpreter.h"
#include "tensorflow/lite/schema/schema_generated.h"
#include "tensorflow/lite/version.h"
```


- 주요 변수 초기화
 - 위에서 살펴본 테스트 프로그램에서 사용한 것과 유사

 - namespace로 묶여 있어서 main_functions.cc의 어느 곳에서나 접근할 수 있지만 다른 파일에서는 접근할 수 없음. 두 개의 서로 다른 파일이 동일한 이름의 변수를 정의할 때 생기는 문제를 방지할 수 있음

 - inference_count 변수: 추론 횟수를 저장하는 변수


```cpp
// Globals, used for compatibility with Arduino-style sketches.
namespace {
tflite::ErrorReporter* error_reporter = nullptr;
const tflite::Model* model = nullptr;
tflite::MicroInterpreter* interpreter = nullptr;
TfLiteTensor* input = nullptr;
TfLiteTensor* output = nullptr;
int inference_count = 0;

// Create an area of memory to use for input, output, and intermediate arrays.
// Finding the minimum value for your model may require some trial and error.
constexpr int kTensorArenaSize = 2 * 1024;
uint8_t tensor_arena[kTensorArenaSize];
}  // namespace
```


- setup() 함수
  - 디버그용 메시지 로깅을 위한 ErrorReporter 객체 생성
  - 모델 객체 생성
  - 모델에서 사용할 Op 접근을 위한 AllOpsResolver 객체 생성
  - 모델 실행을 위한 MicroInterpreter 객체 생성
  - 모델에서 사용될 텐서를 tensor arena의 메모리에 할당
  - 입력, 출력 텐서에 대한 포인터 가져옴
  - 추론 횟수를 저장할 변수 0으로 설정


```cpp
// The name of this function is important for Arduino compatibility.
void setup() {
  // Set up logging. Google style is to avoid globals or statics because of
  // lifetime uncertainty, but since this has a trivial destructor it's okay.
  // NOLINTNEXTLINE(runtime-global-variables)
  static tflite::MicroErrorReporter micro_error_reporter;
  error_reporter = &micro_error_reporter;

  // Map the model into a usable data structure. This doesn't involve any
  // copying or parsing, it's a very lightweight operation.
  model = tflite::GetModel(g_sine_model_data);
  if (model->version() != TFLITE_SCHEMA_VERSION) {
    TF_LITE_REPORT_ERROR(error_reporter,
                         "Model provided is schema version %d not equal "
                         "to supported version %d.",
                         model->version(), TFLITE_SCHEMA_VERSION);
    return;
  }

  // This pulls in all the operation implementations we need.
  // NOLINTNEXTLINE(runtime-global-variables)
  static tflite::ops::micro::AllOpsResolver resolver;

  // Build an interpreter to run the model with.
  static tflite::MicroInterpreter static_interpreter(
      model, resolver, tensor_arena, kTensorArenaSize, error_reporter);
  interpreter = &static_interpreter;

  // Allocate memory from the tensor_arena for the model's tensors.
  TfLiteStatus allocate_status = interpreter->AllocateTensors();
  if (allocate_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter, "AllocateTensors() failed");
    return;
  }

  // Obtain pointers to the model's input and output tensors.
  input = interpreter->input(0);
  output = interpreter->output(0);

  // Keep track of how many inferences we have performed.
  inference_count = 0;
}
```


- loop() 함수
  - 모델 입력으로 사용할 x 값 계산: kXrange, kInferencesPerCycle은 constants.h에 정의된 상수. kXrange는 x의 가능한 최대값을 2파이로 정의. kInferencesPerCycle은 x 값이 0-2파이까지 변하는 한 사이클 동안 수행할 추론 횟수 정의
  - 추론 실행: MicroInterpreter의 Invoke() 메소드 호출
  - 출력 처리: HandleOutput() 함수 호출
  - inference_count 변수 업데이트


```cpp
// The name of this function is important for Arduino compatibility.
void loop() {
  // Calculate an x value to feed into the model. We compare the current
  // inference_count to the number of inferences per cycle to determine
  // our position within the range of possible x values the model was
  // trained on, and use this to calculate a value.
  float position = static_cast<float>(inference_count) /
                   static_cast<float>(kInferencesPerCycle);
  float x_val = position * kXrange;

  // Place our calculated x value in the model's input tensor
  input->data.f[0] = x_val;

  // Run inference, and report any error
  TfLiteStatus invoke_status = interpreter->Invoke();
  if (invoke_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter, "Invoke failed on x_val: %f\n",
                         static_cast<double>(x_val));
    return;
  }

  // Read the predicted y value from the model's output tensor
  float y_val = output->data.f[0];

  // Output the results. A custom HandleOutput function can be implemented
  // for each supported hardware target.
  HandleOutput(error_reporter, x_val, y_val);

  // Increment the inference_counter, and reset it if we have reached
  // the total number per cycle
  inference_count += 1;
  if (inference_count >= kInferencesPerCycle) inference_count = 0;
}
```



### 6.3.3 output_handler.cc

- 주요 기능
  - ErrorReporter 인스턴스를 사용하여 x, y 값을 기록
  - 컴퓨터에서 실행 시 화면에 값이 출력됨


```cpp
#include "tensorflow/lite/micro/examples/hello_world/output_handler.h"

void HandleOutput(tflite::ErrorReporter* error_reporter, float x_value,
                  float y_value) {
  // Log the current X and Y values
  TF_LITE_REPORT_ERROR(error_reporter, "x_value: %f, y_value: %f\n",
                       static_cast<double>(x_value),
                       static_cast<double>(y_value));
}
```

- 아두이노에서 프로그램이 실행되는 경우 arduino_output_handler.cpp 파일로 대체됨

### 6.3.4 애플리케이션 실행하기

#### Hello World 예제 빌드

- 이전에 살펴본 테스트 프로그램과 마찬가지로 make 명령어를 이용하여 빌드
 - TFLM 코드가 다운로드된 상태이어야 함 (git clone 사용)

In [9]:
%cd /content/tensorflow-lite/

! make -f tensorflow/lite/micro/tools/make/Makefile hello_world

/content/tensorflow-lite
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/hello_world/main.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/hello_world/main.o
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/hello_world/main_functions.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/hello_world/main_functions.o
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_L

#### 애플리케이션 실행

- 프로그램은 입력 x에 대한 sin(x) 예측값 y를 계속 출력함

 - output_handler.cc의 HandleOutput() 함수로 작성된 로그
 - 한 번의 추론에 한 줄의 로그가 출력됨

In [10]:
%cd /content/tensorflow-lite/

! tensorflow/lite/micro/tools/make/gen/linux_x86_64/bin/hello_world

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

x_value: 1.7278753*2^1, y_value: -1.4295994*2^-2

x_value: 1.8849551*2^1, y_value: -1.2867725*2^-1

x_value: 1.210171*2^2, y_value: -1.7542461*2^-1

x_value: 1.995567*2^2, y_value: -1.9749510*2^-1

x_value: 1.1780966*2^2, y_value: -1.9622826*2^-1

x_value: 1.2566366*2^2, y_value: -1.8620564*2^-1

x_value: 1.3351763*2^2, y_value: -1.5208751*2^-1

x_value: 1.4137159*2^2, y_value: -1.106089*2^-1

x_value: 1.4922558*2^2, y_value: -1.6822*2^-2

x_value: 1.0*2^-127, y_value: 1.5557474*2^-5

x_value: 1.2566366*2^-2, y_value: 1.2133831*2^-2

x_value: 1.2566366*2^-1, y_value: 1.1161486*2^-1

x_value: 1.8849551*2^-1, y_value: 1.5661856*2^-1

x_value: 1.2566366*2^0, y_value: 1.8288908*2^-1

x_value: 1.5707957*2^0, y_value: 1.9224764*2^-1

x_value: 1.8849551*2^0, y_value: 1.8043818*2^-1

x_value: 1.995567*2^1, y_value: 1.5224055*2^-1

x_value: 1.2566366*2^1, y_value: 1.1094988*2^-1

x_value: 1.4137159*2^1, y_value: 1.374213*2^-2

x_value: 1.5707957*2^1, y_value

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
x_value: 1.7278753*2^1, y_value: -1.4295994*2^-2

x_value: 1.8849551*2^1, y_value: -1.2867725*2^-1

x_value: 1.210171*2^2, y_value: -1.7542461*2^-1

x_value: 1.995567*2^2, y_value: -1.9749510*2^-1

x_value: 1.1780966*2^2, y_value: -1.9622826*2^-1

x_value: 1.2566366*2^2, y_value: -1.8620564*2^-1

x_value: 1.3351763*2^2, y_value: -1.5208751*2^-1

x_value: 1.4137159*2^2, y_value: -1.106089*2^-1

x_value: 1.4922558*2^2, y_value: -1.6822*2^-2

x_value: 1.0*2^-127, y_value: 1.5557474*2^-5

x_value: 1.2566366*2^-2, y_value: 1.2133831*2^-2

x_value: 1.2566366*2^-1, y_value: 1.1161486*2^-1

x_value: 1.8849551*2^-1, y_value: 1.5661856*2^-1

x_value: 1.2566366*2^0, y_value: 1.8288908*2^-1

x_value: 1.5707957*2^0, y_value: 1.9224764*2^-1

x_value: 1.8849551*2^0, y_value: 1.8043818*2^-1

x_value: 1.995567*2^1, y_value: 1.5224055*2^-1

x_value: 1.2566366*2^1, y_value: 1.1094988*2^-1

x_value: 1.4137159*2^1, y_value: 1.374213*2^-2

x_value: 1.5707957*2^1, y_value:

---
## 마이크로컨트롤러에 애플리케이션 배포하기

---

- Microcontroller 또는 MCU (microcontroller unit)
  - 집적 회로 안에 마이크로프로세서와 입출력 모듈, 메모리 등의 컴퓨팅 요소를 내장한 초소형 컴퓨팅 장치
  - 보통 MCU는 임베디드 애플리케이션 용으로 디자인된 연산 장치이며 임베디드 시스템에 널리 사용됨

  - [SparkFun Edge Development Board](https://www.sparkfun.com/products/15170)

  - [Arduino Nano 33 BLE Sense](https://store.arduino.cc/usa/nano-33-ble-sense)

  
- 우리가 사용할 아두이노 보드는 Arduino Nano 33 BLE Sense
  - 아두이노 보드의 종류는 매우 다양하고 그 성능도 제각각이며, 모든 아두이노가 마이크로컨트롤러용 텐서플로우 라이트를 실행할 수 있는 것은 아님
  - Arduino Nano 33 BLE Sense는 텐서플로우 라이트와 호환되는 것 외에도 마이크와 가속도계가 포함되어 있음
  - 아두이노 보드에는 LED가 내장되어 있으며 LED는 사인 값을 시각적으로 출력하는데 사용됨

## 6.4 개발 환경

### Arduino IDE 이용
- Arduino IDE 설치
  - https://www.arduino.cc/en/software  

- **Arduino_TensorFlowLite 라이브러리 설치**
  - Arduino 용 TensorFlowLite 라이브러리
  - 아래 링크에서 zip 파일을 다운로드 받아서 설치
  - https://drive.google.com/file/d/1H8K7CholxtgJ7q4FMpZZ7lFIayOcCO1c/view?usp=sharing
  - 아두이노IDE 메뉴 Sketch(스케치) > Include Library (라이브러리 포함하기) > Add .Zip Library... (.zip 라이브러리 추가...)
  - 설치된 라이브러리 폴더 위치: [사용자 문서 폴더]/Arduino/libraries
  - 보통 Arduino 라이브러리는 Arduino IDE의 라이브러리 매니저를 이용하여 설치하는 것이 일반적인 방식이나 현재 라이브러리 매니저에서 제거된 상태


- **Arduino Nano 33 BLE Sense 보드 설치**
  - Arduino IDE 실행 후 툴 메뉴 클릭 >> 보드 선택 >> 보드 매니저... 클릭 >> 검색 창에서 nano 33 검색 >> Arduino Mbed OS Nano Boards 버전 선택 후 설치




## 6.5 예제 실행하기

- 아두이노용 프로젝트를 빌드하고 장치에 배포하기
- 필요한 것
  - 아두이노 나노 33 BLE 센스
  - USB 케이블
  - Arduino IDE

#### **실행 절차**
- Arduino_TensorFlowLite 라이브러리 예제 로드
  - Arduino IDE에 Arduino_TensorFlowLite 라이브러리가 설치되어 있어야 함
  - 파일 메뉴 >> 예제 >> Arduino_TensorFlowLite >> hello_world 선택
- USB 케이블로 아두이노를 컴퓨터에 연결
- Arduino IDE에서 해당 보드(Arduino Nano 33 BLE) 선택
  - 툴 메뉴 >> 보드 (Board)
- 포트 선택
  - 툴 메뉴 >> 포트 (Port)
- Arduino IDE에서 업로드 버튼(오른쪽 방향 화살표) 클릭
  - 코드가 컴파일되고 완료되면 보드에 업로드되어 실행됨
- 업로드가 성공적으로 완료되면 LED가 점멸됨
  - 주황 LED가 주기적으로 밝기가 변함
  - 툴 메뉴 >> 시리얼 플로터 혹은 시리얼 모니터를 선택하여 시간이 지남에 따라 변화하는 값을 확인할 수 있음





#### 아두이노 예제 코드의 차이점
- Arduino IDE에서 hello_world 예제를 로드하면 관련 파일들이 열리는데 github 저장소의 코드와 약간의 차이점이 있음

- 소스 파일 확장자
 - .cc 파일은 .cpp 파일로 변경

- main_functions.cc 파일은 hello_world 파일이 됨

- constants.cc, main.cc, output_handler.cc 파일은 파일명 앞에 arduino가 추가되어 arduino_constants.cpp, arduino_main.cpp, arduino_output_handler.cpp 파일이 됨

- model.cpp, model.h: 모델 데이터 파일

### 6.5.1 아두이노 출력 다루기

- 모델의 출력 결과에 따라 아두이노의 내장 LED를 제어
 - 입력 x는 0에서 2파이까지 변하므로 모델의 출력 y는 대략적으로 -1에서 1사이 값이 될 것임
 - -1인 경우 밝기를 최저로 1인 경우 밝기를 최대로 하도록 구현됨

- arduino_output_handler.cpp 파일의 HandleOutput() 함수에서 처리
 - brightness 변수 값을 계산하고 이를 analogWrite() 함수에서 사용함
 - 아두이노 내장 LED 핀에서 PWM을 지원하지 않으면 단순 on/off 만 가능
 - ErrorReporter를 통해서 시리얼 포트를 이용해 데이터를 기록할 수 있음. 이를 Arduino IDE 툴 메뉴의 시리얼 모니터 혹은 시리얼 플로터를 이용하여 확인할 수 있음
 - 시리얼 통신: 마이크로컨트롤러가 호스트 컴퓨터와 통신하는 일반적인 방법이며 디버깅에 흔히 사용됨
 - 시리얼 모니터나 시리얼 플로터는 시리얼 포트를 통해 수신된 데이터를 캡처하고 표시하기 위한 도구

```cpp
#include "output_handler.h"

#include "Arduino.h"
#include "constants.h"

// The pin of the Arduino's built-in LED
int led = LED_BUILTIN;

// Track whether the function has run at least once
bool initialized = false;

// Animates a dot across the screen to represent the current x and y values
void HandleOutput(tflite::ErrorReporter* error_reporter, float x_value,
                  float y_value) {
  // Do this only once
  if (!initialized) {
    // Set the LED pin to output
    pinMode(led, OUTPUT);
    initialized = true;
  }

  // Calculate the brightness of the LED such that y=-1 is fully off
  // and y=1 is fully on. The LED's brightness can range from 0-255.
  int brightness = (int)(127.5f * (y_value + 1));

  // Set the brightness of the LED. If the specified pin does not support PWM,
  // this will result in the LED being on when y > 127, off otherwise.
  analogWrite(led, brightness);

  // Log the current brightness value for display in the Arduino plotter
  TF_LITE_REPORT_ERROR(error_reporter, "%d\n", brightness);
}
```





### 6.5.2 커스텀 버전 만들기

- LED 점등 주기 변경
  - arduino_constants.cpp 파일에서 kInferencesPerCycle 상수 값을 조정하여 밝기 변화 주기를 변경할 수 있음

  - 변화 속도를 더 빠르게 하거나 더 느리게 할 수 있음

